In [ ]:
# !pip install plotly

In [1]:
import plotly.graph_objects as go
import pandas as pd

def classify_sentiment(score):
    if score > 0.05:
        return 'Positive'
    if score < -0.05:
        return 'Negative'
    return 'Neutral'


In [2]:
def prep_sentiment_dataset(df):
    df['post_date'] = 1000 * df['post_date']
    df['date'] = pd.to_datetime(df['post_date'], unit='ms')
    df = df[['date', 'Sentiment']].copy()
    df['Sentiment Classification'] = df['Sentiment'].apply(classify_sentiment)
    df = df.loc[df['Sentiment Classification'] != 'Neutral'].copy()
    df_pos = df.loc[df['Sentiment Classification'] == 'Positive'].copy()
    df_neg = df.loc[df['Sentiment Classification'] == 'Negative'].copy()

    df_pos['Count'] = 1
    df_pos = df_pos.resample('D', on='date').sum()
    df_pos.reset_index(inplace=True)
    df_pos = df_pos[['date', 'Count']].copy()
    df_pos.set_index('date', inplace=True)

    df_neg['Count'] = -1
    df_neg = df_neg.resample('D', on='date').sum()
    df_neg.reset_index(inplace=True)
    df_neg = df_neg[['date', 'Count']].copy()
    df_neg.set_index('date', inplace=True)

    df_combined = df_pos.join(df_neg, how='outer', lsuffix='Pos', rsuffix='Neg')
    df_combined.reset_index(inplace=True)
    df_combined.columns = ['Date', 'Positive Tweets', 'Negative Tweets']
    df_combined['Zero'] = 0
    return df_combined

In [3]:
dfs = [
    ['AMZN', pd.read_csv('./data/sent-amazon.csv')],
    ['APPL', pd.read_csv('./data/sent-apple.csv')],
    ['GOOG', pd.read_csv('./data/sent-google.csv')],
    ['MSFT', pd.read_csv('./data/sent-microsoft.csv')],
    ['TSLA', pd.read_csv('./data/sent-tesla.csv')]
]

dfs_processed = []

for ticker, df in dfs:
    new_df = prep_sentiment_dataset(df)
    new_df['Ticker'] = ticker
    dfs_processed.append(new_df)

sentiment_all = pd.concat(dfs_processed)
sentiment_all.to_csv('./data/daily-sentiment-summary.csv', index=False)
sentiment_all

,Date,Positive Tweets,Negative Tweets,Zero,Ticker
0,2015-01-01,42,-61,0,AMZN
1,2015-01-02,76,-34,0,AMZN
2,2015-01-03,28,-8,0,AMZN
3,2015-01-04,29,-10,0,AMZN
4,2015-01-05,120,-29,0,AMZN
...,...,...,...,...,...
1821,2019-12-27,556,-300,0,TSLA
1822,2019-12-28,318,-139,0,TSLA
1823,2019-12-29,291,-174,0,TSLA
1824,2019-12-30,555,-353,0,TSLA


In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df = pd.read_csv('./data/tsla_stock_data.csv')

tesla = sentiment_all.loc[sentiment_all['Ticker'] == 'TSLA'].copy()

fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=("Price", 'Sentiment'),
            shared_xaxes=True,
            vertical_spacing =0.3)

# Add a line chart
fig.add_trace(go.Line(x=df['date'],
                       y=df['close'],
                       name='Close'), row=1, col=1)

fig.add_trace(go.Candlestick(
    x=tesla['Date'],
    open=tesla['Zero'],
    high=tesla['Positive Tweets'],
    low=tesla['Negative Tweets'],
    close=tesla['Zero']
), row=2, col=1)

# Update the layout
fig.update_layout(title='Title',
                  xaxis_title='Date',
                  yaxis_title='Price / Volume')

# Show the chart
fig.show()

c:\Users\fuze0\anaconda3\envs\dev\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


